In [ ]:
import pandas as pd
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

en_zh_translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-zh", device=device)
zh_en_translator = pipeline("translation", model="Helsinki-NLP/opus-mt-zh-en", device=device)

df_train = pd.read_csv('train.csv', encoding='latin1')
df_test = pd.read_csv('test.csv', encoding='latin1')

def translate_and_save(df, filename):
    df['text'] = df['text'].astype(str)
    # Translate English text to Chinese
    zh_translated = df['text'].apply(lambda x: en_zh_translator(x, clean_up_tokenization_spaces=True)[0]['translation_text'])

    # Translate the Chinese text back to English
    en_back_translated = zh_translated.apply(lambda x: zh_en_translator(x, clean_up_tokenization_spaces=True)[0]['translation_text'])

    df_translated = pd.DataFrame({
        'text': en_back_translated,
        'sentiment': df['sentiment']
    })

    df_augmented = pd.concat([df[['text', 'sentiment']], df_translated], ignore_index=True)

    df_augmented.to_csv(filename, index=False)

translate_and_save(df_train, 'train_augmented.csv')
translate_and_save(df_test, 'test_augmented.csv')

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activ

In [6]:
!pip install sentencepiece


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
